In [ ]:
# default_exp monitoring

# Monitoring
In this notebook, we monitor our pipeline efficiency.

In [ ]:
%load_ext autoreload
%autoreload 2
from IPython.display import Video

In [ ]:
#export
%load_ext autoreload
%autoreload 2
from trr265.data_provider import DataProvider

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
dp = DataProvider('/Users/hilmarzech/Projects/trr265/trr265/data/')

In [ ]:
phone = dp.get_phone_data(update = True).query("(screened_for_b07==False)")

Preparing phone_data


In [ ]:
ba = dp.get_ba_data().query("redcap_event_name=='erhebungszeitpunkt_arm_1'")

In [ ]:
mov_data = dp.get_mov_data(update = True)

Preparing mov_data
Participants: b186, d092, d140, d146, d147, d153, m158, m238, m257 have no associated redcap IDs and are excluded from the following analyses.


In [ ]:
ba.query('(bx_smartphone==bx_smartphone) and (center=="d")').participant_id

473      211
483      213
588      227
598      233
635      246
        ... 
4249    1794
4329    1804
4371    1805
4392    1809
4525    1903
Name: participant_id, Length: 114, dtype: object

In [ ]:
len(mov_data[mov_data['participant'].str.startswith('d')].participant.unique())

106

In [ ]:
phone['aud_tested'] = phone.participant_id.isin(ba.participant_id.unique())

In [ ]:
def count_participants(df, filt):
    return len(df.query(filt).participant_id.unique())

def make_count_dict(df):
    included = "(screen_incl==1 and screen_excl_cons!=screen_excl_cons)"
    ru_incl = "(screen_excl_cons==0)"
    filters = {"included": included,
    "not_included": "(screen_incl==0 and screen_excl_cons!=screen_excl_cons)",
    "rucksprache": "(screen_incl==2 or screen_excl_cons==screen_excl_cons)",
    "missing": "(screen_incl!=screen_incl)",
    "ru_excl":  "(screen_excl_cons==1)",
    "ru_incl": ru_incl,
    "ru_missing": "(screen_included==2 and screen_excl_cons!=screen_excl_cons)",
    "incl_aud": "(%s and screen_group=='AUD')"%included,
    "incl_con": "(%s and screen_group=='Control')"%included,
    "ru_in_aud": "(%s and screen_group=='AUD')"%ru_incl,
    "ru_in_con": "(%s and screen_group=='Control')"%ru_incl,
    "aud_tested": "(screen_group=='AUD' and tested==True)"}
    
    count_dict = {}
    for key, value in filters.items():
        count_dict[key] = count_participants(df, value)
    return count_dict
        
count_dict = make_count_dict(data)

In [ ]:
def get_data():
    phone_data = dp.get_phone_data()
    redcap_data = dp.get_ba_data()#load_redcap_data()
    redcap_data['event'] = 'ba'
    print('a')
    # One MRT questions (Schleifarbeiten) is asked twice with the same label. We relabel it.
    phone_data.rename(columns={'bx_mrt_03':'bx_mrt_03_screening'}, inplace=True)
    # We also relable piped variables
    piped_dict = {c:c+"_piped" for c in list(redcap_data.columns) if c.startswith('screen') or c in ["study_id","center"]}
    redcap_data.rename(columns=piped_dict , inplace=True)
    overlapping = set(phone_data.columns).intersection(set(redcap_data.columns))-{'participant_id'}
    # Checking for overlapping variables
    print('b')
    assert not overlapping, "Overlapping variables in merge: %s. Adjust in piped_dict?"%', '.join(list(overlapping))
    phone_data['participant_id'] = phone_data.participant_id.astype(str)
    redcap_data['participant_id'] = redcap_data.participant_id.astype(str)
    print('o')
    df = pd.merge(phone_data, redcap_data, how="outer", on=["participant_id"], indicator = True, validate='one_to_many')
    print('c')
    df['phone_screened'] = df._merge != "right_only"
    print('d')
    df['tested'] = df._merge != "left_only"
    print('e')
    df['event'].fillna("phone", inplace = True) # Adding phone event
    print('f')
    df.drop(columns='_merge', inplace = True)
    print(1)
    
    
    # GENERATING VARIABLES
    
    df['has_phone'] = (df.mov_id.isin(["dnan","mnan","bnan"]) | df.mov_id.isna())==False
    
    # Assigned screening groups
    df['screen_group_as'] = np.nan
    df.loc[df.screen_aud_group3==1, 'screen_group_as'] = "No group" # no group
    df.loc[df.screen_aud_group1==1, 'screen_group_as'] = "Control" # control
    df.loc[df.screen_aud_group2==1, 'screen_group_as'] = "AUD" # AUD
    # Calculated groups
    df['screen_group_calc'] = np.nan
    df.loc[df.screen_aud_nogroup==1, 'screen_group_calc'] = "No group" # no group
    df.loc[df.screen_aud_control==1, 'screen_group_calc'] = "Control" # control
    df.loc[df.screen_aud_aud==1, 'screen_group_calc'] = "AUD" # AUD
    # Filling in missing assigned
    df['screen_group'] = df.screen_group_as.fillna(df.screen_group_calc)
    print(2)
    # Assigned BA groups
    df['bx_group_as'] = np.nan
    df.loc[df.bx_aud_group3==1, 'bx_group_as'] = "No group" # no group
    df.loc[df.bx_aud_group1==1, 'bx_group_as'] = "Control" # control
    df.loc[df.bx_aud_group2==1, 'bx_group_as'] = "AUD" # AUD
    # Calculated groups
    df['screen_group_calc'] = np.nan
    df.loc[df.bx_aud_nogroup==1, 'bx_group_calc'] = "No group" # no group
    df.loc[df.bx_aud_control==1, 'bx_group_calc'] = "Control" # control
    df.loc[df.bx_aud_aud==1, 'bx_group_calc'] = "AUD" # AUD
    # Filling in missing assigned
    df['bx_group'] = df.bx_group_as.fillna(df.bx_group_calc)
    
    # Generating screen_included variable
    df['screen_included'] = np.nan
    # Initially set to consult included
    df.loc[df.screen_excl_cons == 0, 'screen_included'] = True
    # Fill missings with SHKs decision
    df['screen_included'] = df['screen_included'].fillna(df.screen_incl)
    # Participants that show up in B0 are also tagged as included
    df.loc[df.tested,'screen_included'] = True
    print(3)

    
    # Filters
    df['is_s01'] = (df.screened_for_s01==True)# | (df.bx_is_s01==True)
    
    # CHECKS
    print("Participants are in redcap, but not in phone screening: %s"%', '.join(map(str,list(df[(df.phone_screened == False) & (df.tested==True)].participant_id.unique()))))
    # Participants with no BA
    has_ba = df[(df.tested)].groupby('participant_id').event.agg(lambda x: (x=='ba').mean()>0)
    no_ba = has_ba[has_ba==False].index
    print("\nParticipants are in later sessions but not in BA: %s"%', '.join(map(str,no_ba)))
    return df
import pandas as pd
import numpy as np
data = get_data()

In [ ]:
phone.query("screen_date>'2021-06-29'").query("(center=='d')")

In [ ]:
phone.groupby('center').participant_id.count()

## Getting the data

In [ ]:
def get_sankey_parameter(df):
    count_dict = make_count_dict(df)
    print('''
    Teilnehmer [%(included)d] Included 
    Teilnehmer [%(not_included)d] Not Included 
    Teilnehmer [%(rucksprache)d] Rücksprache 
    Teilnehmer [%(missing)d] Missing 
    Rücksprache [%(ru_excl)d] Rü Excluded
    Rücksprache  [%(ru_incl)d] Rü Included
    Rücksprache [%(ru_missing)d] Rü Missing
    Included [%(incl_aud)d] Direkt AUD
    Included [%(incl_con)d] Control
    Rü Included [%(ru_in_con)d] Control
    Rü Included [%(ru_in_aud)d] AUD Gesamt
    Direkt AUD [%(aud_tested)d] AUD Gesamt
    '''%count_dict)

## Dresden

### Dresden before July

In [ ]:
get_sankey_parameter(data.query("(center=='d') and (screen_date<'2021-07-01')"))


    Teilnehmer [443] Included 
    Teilnehmer [153] Not Included 
    Teilnehmer [283] Rücksprache 
    Teilnehmer [70] Missing 
    Rücksprache [55] Rü Excluded
    Rücksprache  [157] Rü Included
    Rücksprache [56] Rü Missing
    Included [127] Direkt AUD
    Included [313] Control
    Rü Included [43] Control
    Rü Included [109] AUD Gesamt
    Direkt AUD [150] AUD Gesamt
    


In [ ]:
%%html
<iframe src="https://sankeymatic.com/build/" width="1200" height="1000"></iframe>

<img width="700" height="118" src='images/sankey_dresden_before_july.png' />

Screened: 77 of 949 (8%) included.

BA: 77 of 260 (30%) included.

### After July

In [ ]:
get_sankey_parameter(data.query("(center=='d') and (screen_date>'2021-07-01')"))


    Teilnehmer [66] Included 
    Teilnehmer [62] Not Included 
    Teilnehmer [58] Rücksprache 
    Teilnehmer [50] Missing 
    Rücksprache [10] Rü Excluded
    Rücksprache  [19] Rü Included
    Rücksprache [25] Rü Missing
    Included [41] Direkt AUD
    Included [25] Control
    Rü Included [2] Control
    Rü Included [17] AUD Gesamt
    Direkt AUD [35] AUD Gesamt
    


<img width="700" height="118" src='images/sankey_dresden_after_july.png' />

In [ ]:
%%html
<iframe src="https://sankeymatic.com/build/" width="1200" height="1000"></iframe>


Screened: 45 of 236 (19%) included

BA: 45 of 52 (86%) included.

## Mannheim

In [ ]:
get_sankey_parameter(data.query("(center=='m')"))


    Teilnehmer [287] Included 
    Teilnehmer [89] Not Included 
    Teilnehmer [230] Rücksprache 
    Teilnehmer [114] Missing 
    Rücksprache [43] Rü Excluded
    Rücksprache  [144] Rü Included
    Rücksprache [34] Rü Missing
    Included [210] Direkt AUD
    Included [75] Control
    Rü Included [20] Control
    Rü Included [123] AUD Gesamt
    Direkt AUD [224] AUD Gesamt
    


In [ ]:
%%html
<iframe src="https://sankeymatic.com/build/" width="1200" height="1000"></iframe>

<img width="1000" height="360" src='images/sankey_mannheim.png' />

Screened: 211 of 720 (29%) included.

BA: 211 of 347 (60%) included.

## Berlin

In [ ]:
get_sankey_parameter(data.query("(center=='b')"))


    Teilnehmer [325] Included 
    Teilnehmer [164] Not Included 
    Teilnehmer [290] Rücksprache 
    Teilnehmer [318] Missing 
    Rücksprache [30] Rü Excluded
    Rücksprache  [87] Rü Included
    Rücksprache [124] Rü Missing
    Included [175] Direkt AUD
    Included [108] Control
    Rü Included [19] Control
    Rü Included [59] AUD Gesamt
    Direkt AUD [196] AUD Gesamt
    


In [ ]:
%%html
<iframe src="https://sankeymatic.com/build/" width="1200" height="1000"></iframe>

<img width="1000" height="510" src='images/sankey_berlin.png' />

Screened: 166 of 1097 (15%) included.

BA: 166 of 347 (65%) included.

In [ ]:
#export
@patch
def decode_wm_strings(self:WMDataProvider, gbe_data):
    df = self.decode_gbe_strings(gbe_data, 'WorkingMemoryGame')
    df['success'] = df.success.astype(int)
    return df

In [ ]:
gbe_data = dp.get_gbe_data()
df = dp.decode_wm_strings(gbe_data)

> The raw data gives us trial ids.  However, the different trial types (no distractor, encoding distractor, delayed distractor) and levels (2 to 10) are not yet shown. 

In [ ]:
df[['gbe_index','trial_number','timestarted','timesubmitted','timetaken','trialid','trialrot','success']]

,gbe_index,trial_number,timestarted,timesubmitted,timetaken,trialid,trialrot,success
0,b001_001,1,2020-02-24 20:48:52.812,2020-02-24 20:52:50.167,1.506,420,1,1
1,b001_001,2,2020-02-24 20:48:52.812,2020-02-24 20:52:50.167,1.471,379,1,1
2,b001_001,3,2020-02-24 20:48:52.812,2020-02-24 20:52:50.167,1.154,343,1,1
3,b001_001,4,2020-02-24 20:48:52.812,2020-02-24 20:52:50.167,2.004,463,0,1
4,b001_001,5,2020-02-24 20:48:52.812,2020-02-24 20:52:50.167,1.589,616,3,1
...,...,...,...,...,...,...,...,...
82226,m271_010,28,2021-07-21 10:25:34.373,2021-07-21 10:29:45.451,0.613,1646,1,0
82227,m271_010,29,2021-07-21 10:25:34.373,2021-07-21 10:29:45.451,2.297,2499,0,0
82228,m271_010,30,2021-07-21 10:25:34.373,2021-07-21 10:29:45.451,2.573,1644,3,0
82229,m271_010,31,2021-07-21 10:25:34.373,2021-07-21 10:29:45.451,3.638,2586,2,1


### Adding trial types and levels
> Here, werial types based on the app specifications.  We also double check that everything was specified correctly, by comparing the trial types with the app resources. add the t

In [ ]:
#export
@patch
@get_efficiently
def get_wm_trial_types(self:WMDataProvider):
    '''This checks that each trial type follows the appropriate specifications.'''
    types = open(os.path.join(self.external, "types.xml"),"r", encoding='UTF-8').read().encode('utf-8')
    types_dict = xmltodict.parse(types)
    type_list = types_dict['trials']['trialtype']
    type_df = []
    for i, t in enumerate(type_list):
        # Getting the type and difficulty
        trial_type = t['@type']
        level = int(t['@difficulty'])
        if level > 10:
            continue
        # Geting individual trial specifications for each type
        for trial in t['trial']:
            trial_dict = {}
            trial_dict['trialid'] = trial['@id'] #id
            trial_dict['trial_type'] = trial_type #
            trial_dict['level'] = level

            # Getting additional information

            board = trial['board']
            # Checking if circles were shown delayed
            if type(board) == collections.OrderedDict:
                board = [board]
            trial_dict['trial_boards'] = []
            # Checking if the trial had distractors
            has_distractor = False
            trial_dict['number_of_boards'] = 0
            for b in board:
                try:
                    if '#text' in b.keys():
                        trial_dict['trial_boards'].append(b['#text'])
                        trial_dict['number_of_boards'] += 1
                    if '#text' in b.keys() and 'D' in b['#text']:
                        has_distractor = True
                        break
                except:
                    print(trial_dict, board)
            is_delayed = int(trial_dict['number_of_boards'] > 1)
            trial_dict['has_distractor'] = int(has_distractor)
            trial_dict['is_delayed'] = int(trial_dict['number_of_boards'] > 1)
            type_df.append(pd.Series(trial_dict))
    type_df = pd.DataFrame(type_df)
    type_df.trial_type.replace({'0':'no_distractor_1',
                               '1':'encoding_distractor',
                               '2':'delayed_distractor',
                               '3':'no_distractor_2'}, inplace = True)
    return type_df

> Checking if each trial type is specified correctly:

In [ ]:
type_df = dp.get_wm_trial_types()
type_df.groupby(['trial_type'])[['has_distractor','is_delayed']].mean().astype(bool)

,has_distractor,is_delayed
trial_type,,
4,False,True
5,True,True
delayed_distractor,True,True
encoding_distractor,True,False
no_distractor_1,False,False
no_distractor_2,False,False


> Adding trial types and levels to dataframe.

In [ ]:
#export
@patch
def add_trial_types(self:WMDataProvider, df):
    type_df = self.get_wm_trial_types()
    if 'trial_type' in df.columns:
        df = df.drop.columns('trial_type')
    df = df.merge(type_df[['trialid','trial_type','level']], on = 'trialid', how = 'left', validate = 'many_to_one')
    return df
    

In [ ]:
df = dp.add_trial_types(df)

#### Checking trial types in dataset

In [ ]:
df.trial_type.value_counts()

no_distractor_1        20554
encoding_distractor    20510
no_distractor_2        20493
delayed_distractor     20284
5                        195
4                        195
Name: trial_type, dtype: int64

> Warning: Note that some participants completed old versions of the task (as some trials were associated with trial types 4 and 5).

### Getting the data

In [ ]:
#export
@patch
@get_efficiently
def get_wm_data(self:WMDataProvider):
    gbe_data = self.get_gbe_data()
    df = self.decode_wm_strings(gbe_data)
    df = self.add_trial_types(df)
    return df